# Exercise — Oil & Gas Production Allocation

## Scenario

An operator manages three wells: `A`, `B`, and `C`.
Each well has:
- a maximum daily production capacity (barrel/day)
- a minimum “contracted” production they should try to meet if possible

There is also a shared pipeline with a maximum daily throughput.

Each well has a different revenue per barrel due to quality differences.

You must decide the production per well to maximise revenue, subject to:
- well capacity constraints
- pipeline capacity
- contracted minimums

### Inputs
You are given this data as Python dictionaries:
```python
well_capacities = {
    "A": {"min": 200, "max": 600},
    "B": {"min": 100, "max": 500},
    "C": {"min":  50, "max": 300},
}

pipeline_capacity = 1000  # barrels per day

revenue_per_barrel = {
    "A": 45.0,
    "B": 40.0,
    "C": 50.0,
}
```
### Task
Implement:
```python
from typing import Dict

def optimise_production_plan(
    well_capacities: Dict[str, Dict[str, float]],
    pipeline_capacity: float,
    revenue_per_barrel: Dict[str, float],
) -> Dict[str, float]:
    """
    Compute the optimal production per well subject to capacity and pipeline constraints.
    """
    ...
```
### Requirements
- Formulate and solve a linear program using `scipy.optimize.linprog`.
- Decision variables: production of each well (`A`, `B`, `C`).
- Constraints:
    - For each well: min $\leq$ production $\leq$ max
    - Sum of all production $\leq$ pipeline capacity
- Objective: maximize tota revenue
- Return a dictionary
```python
{
  "A": float,
  "B": float,
  "C": float,
  "total_revenue": float
}
```
- Handle potential solver failure by raising a clear error or returning a meaningful message. 
    



## Thinking through the problem

### Define decision variables

We need variables that represent the choices we control.

Let: 
- $x_A$ be the production of well A (per day)
- $x_B$ be the production of well B
- $x_C$ be the production of well C

So our decision vector is:
$$
x = [x_A, x_B, x_C]
$$

### Define objective in math

$$
\text{Revenue}= 45x_A + 40x_B + 50x_C
$$

In vector form:
$$
\text{max}\;c^T x
$$
with $c = [45, 40, 50]$.

But `linprog` minimizes. So we pass `c = [-45, -40, -50]` and then negate the result.

### Write the constraints in math

We have three types of constraints.

a) Pipeline capacity
Sum of all production must not exceed the pipeline capacity:
$$
x_A + x_B + x_C \leq pipeline\_capacity
$$

In vector/matrix form:
- $A_{ub} = [1, 1, 1]$
- $b_{ub} = pipeline\_capacity$

b) Per-well min and max
For each well:
- $\text{min}_A \leq x_A \leq \text{max}_A$
- $\text{min}_B \leq x_B \leq \text{max}_B$
- $\text{min}_C \leq x_C \leq \text{max}_C$

In `linprog`, these become bounds on each variable:
```python
bounds = [
    (min_A, max_A),
    (min_B, max_B),
    (min_C, max_C)
]
```

We don't need to encode min/max as extra rows in `A_ub`; `bounds` handles it cleanly.

No need for expliciit non-negativity constraints because the lower bounds already enforce that.

### Map this to `linprog` parameters


For SciPy:

```python
from scipy.optimize import linprog

c = [-45, -40, -50] 
A_ub = [[1, 1, 1]]
b_ub = [pipeline_capacity]

bounds = [(min_A, max_A), (min_B, max_B), (min_C, max_C)]

res = linprog(c, A_ub, b_ub, bounds=bounds, method='highs')

Then:
- `res.x` $\rightarrow$ array with $[x_A, x_B, x_C]$
- `res.fun` $\rightarrow$ value of minimized objective = $\Sigma_{i=1}^3 c_i x_i$ = negative revenue
- So total revenue is `-res.fun`



## Code implementation

In [ ]:
from typing import Dict, List, Tuple
from scipy.optimize import linprog

def optimise_production_plan(
    well_capacities: Dict[str, Dict[str, float]],
    pipeline_capacity: float,
    revenue_per_barrel: Dict[str, float],
) -> Dict[str, float]:
    """
    Compute the optimal production per well subject to capacity and pipeline constraints.

    Args: 
        well_capacities: Dict mapping well name -> {"min": float, "max": float}
        pipeline_capacity: Maximum total pipeline throughput (barrels/day)
        revenue_per_barrel: Dict mapping well name -> revenue per barrel
    
    Returns:
        Dict with optimal production per well and total revenue, e.g.:
        {
            "A": 300.0,
            "B": 400.0,
            "C": 300.0,
            "total_revenue": 39500.0
        }
    """
    # ============================================================
    # STEP 1: Establish a deterministic variable ordering
    # ============================================================
    # We need a consistent order for our decision variables (x_A, x_B, x_C)
    # to match the order in our coefficient vectors and constraint matrices.
    # Sorting ensures the same input always produces the same variable ordering,
    # which is important for reproducibility and debugging.
    wells: List[str] = sorted(well_capacities.keys())
    # Example: wells = ["A", "B", "C"] means x[0]=A, x[1]=B, x[2]=C

    # ============================================================
    # STEP 2: Build the objective function coefficients
    # ============================================================
    # Mathematical objective: maximize revenue = 45*x_A + 40*x_B + 50*x_C
    # However, scipy.optimize.linprog MINIMIZES by default.
    # 
    # To maximize revenue, we minimize the negative revenue:
    #   minimize: -45*x_A - 40*x_B - 50*x_C
    # 
    # This is equivalent because:
    #   max(f(x)) = -min(-f(x))
    #
    # The coefficient vector c represents: minimize c^T * x
    # So we negate all revenue values to convert maximization to minimization.
    c = [-revenue_per_barrel[w] for w in wells]
    # Example: If revenue = {"A": 45, "B": 40, "C": 50}
    #          Then c = [-45, -40, -50]

    # ============================================================
    # STEP 3: Build the pipeline capacity constraint
    # ============================================================
    # Constraint: x_A + x_B + x_C <= pipeline_capacity
    # 
    # In matrix form: A_ub * x <= b_ub
    # Where A_ub is a matrix and b_ub is a vector.
    #
    # For our single constraint: [1, 1, 1] * [x_A, x_B, x_C]^T <= pipeline_capacity
    # So A_ub = [[1, 1, 1]] (one row, three columns)
    #    b_ub = [pipeline_capacity]
    A_ub = [[1.0 for _ in wells]]  # One row with 1.0 for each well
    b_ub = [pipeline_capacity]      # Right-hand side of the inequality

    # ============================================================
    # STEP 4: Set variable bounds (per-well min/max constraints)
    # ============================================================
    # For each well w, we have: min_w <= x_w <= max_w
    # 
    # Instead of adding these as separate inequality constraints in A_ub,
    # linprog allows us to specify bounds directly, which is more efficient.
    # 
    # bounds[i] = (lower_bound, upper_bound) for variable x[i]
    # Use None for unbounded (e.g., (0, None) means x >= 0 with no upper bound)
    bounds: List[Tuple[float, float]] = []
    for w in wells:
        w_min = well_capacities[w]["min"]  # Lower bound (contracted minimum)
        w_max = well_capacities[w]["max"]  # Upper bound (well capacity)
        bounds.append((w_min, w_max))
    # Example: bounds = [(200, 600), (100, 500), (50, 300)]
    #          means: 200 <= x_A <= 600, 100 <= x_B <= 500, 50 <= x_C <= 300

    # ============================================================
    # STEP 5: Solve the linear program
    # ============================================================
    # Call the solver with all our parameters:
    # - c: objective coefficients (we're minimizing -revenue)
    # - A_ub, b_ub: inequality constraints (pipeline capacity)
    # - bounds: variable bounds (well min/max)
    # - method='highs': uses the HiGHS solver (fast and reliable)
    res = linprog(
        c,
        A_ub=A_ub,
        b_ub=b_ub,
        bounds=bounds,
        method='highs',
    )

    # Check if the solver found an optimal solution
    # If not, the problem might be infeasible (no solution exists) or unbounded
    if not res.success:
        raise ValueError(f"LP solver failed: {res.message}")

    # ============================================================
    # STEP 6: Extract and format the results
    # ============================================================
    # res.x contains the optimal decision variables in the same order as 'wells'
    # res.fun contains the optimal value of the MINIMIZED objective function
    x_opt = res.x  # Optimal production values: [x_A_opt, x_B_opt, x_C_opt]
    
    # Remember: we minimized -revenue, so res.fun = -actual_revenue
    # Therefore: actual_revenue = -res.fun
    total_revenue = -res.fun

    # Build a readable dictionary mapping well names to their optimal production
    result: Dict[str, float] = {
        w: float(x) for w, x in zip(wells, x_opt)
    }
    result["total_revenue"] = float(total_revenue)
    return result

# ============================================================
# Test algorithm
# ============================================================
# if __name__ == "__main__":    
well_capacities_demo = {
    "A": {"min": 200, "max": 600},
    "B": {"min": 100, "max": 500},
    "C": {"min":  50, "max": 300},
}
pipeline_capacity_demo = 1000
revenue_demo = {"A": 45.0, "B": 40.0, "C": 50.0}

prod_plan = optimise_production_plan(
    well_capacities_demo,
    pipeline_capacity_demo,
    revenue_demo,
)
print("Exercise result:", prod_plan)

Exercise result: {'A': 600.0, 'B': 100.0, 'C': 300.0, 'total_revenue': 46000.0}
